# Kodluyoruz - Carbon Consulting Veri Bilimi Bootcamp'i 4. Hafta Notebook

Bu notebook içerisinde dimensionality reduction tekniklerini ve bunlarin clustering algoritmalari ile kullanımını göreceğiz.

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sklearn

## Numpy

Numpy kütüphanesi genel olarak matematiksel işlemleri hızlandırmak için kullanılıyor. Vektörler üzerine hızlı işlem yapmamızı sağlıyor diyebiliriz.

Numpy ile şunları yapıyor olacağız:
- Boş vektörler ve matrisler oluşturacağız. (0, 1 ve 5 ile dolu)
- Oluşturduğumuz vektör ve matrislerin uzunluklarını/biçimlerini değiştireceğiz.
- Oluşturduğumuz matrislerin belirli elemanlarına erişmeye çalışacağız.
- Ortalama, standart sapma gibi değerleri hesaplayacağız.
- Vektörlerin iç çarpımlarını (dot product) hesaplayacağız.
- Bir vektörün içerisinden rastgele bir eleman seçeceğiz.
- Rastgele bir veri grubu oluşturmaya çalışacağız.

## Matplotlib

Matplotlib kütüphanesi daha çok veri görselleştirme için kullanılıyor. 

Matplotlib ile şunları yapacağız:
- Az önce oluşturmuş olduğumuz veri grubunu görselleştireceğiz.
- Bar chart, scatter plot, line chart,... gibi farklı görselleştirme tiplerini inceleyeceğiz.
- Lejant, ızgara yapıları, figür boyutları ve eksen etiketlerini bir ve birden fazla figür için değiştireceğiz.

## Seaborn

Seaborn kütüphanesi ise yine görselleştirme üzerine kullanılan bir kütüphane. Ayrıca, ekstra olarak içerisinde hazır kullanabileceğimiz bir kaç veri seti de mevcut. Bu yüzden, artık oluşturduğumuz rastgele veri grubu ile değil, hazır verilerle çalışmaya devam edelim.

Seaborn ile şunları yapacağız:
- Yeni bir veri seti indireceğiz ve kullanmaya başlayacağız.
- Bu veri setini görselleştireceğiz. (Sürpriz, veri analizine giriş!)
- Oluşturduğumuz görseller üzerinden veri seti hakkında yorum yapmaya çalışacağız. (Daha önce hiç tanımadığımız bir veriyi anlamladırmaya çalışacağız.)

## Pandas

Pandas kütüphanesini ise elimizdeki veri üzerine daha kolay işlemler yapmak için kullanıyoruz. Sütunlara ya da belirlemiş olduğumuz koşullara göre elimizdeki objeyi güncelleyebiliyor, gruplayabiliyor ve farklı işlemler yapabiliyoruz. Aynı zamanda elimizdeki veri setini hızlıca anlamlandırmak/incelemek için de kullanıyoruz.

Pandas ile şunları yapacağız:
- Bir Pandas objesi oluşturacağız. (DataFrame)
- Excel, CSV, JSON gibi yapılardan veri okuyacağız.
- Kütüphanenin özelliklerini kullanarak veri seti inceleyeceğiz.
- Veri setini gruplayacağız.
- Veri seti üzerinde belirlediğimiz koşullara göre değişiklikler yapacağız.

## Sklearn

Sklearn kütüphanesi ise daha çok istatistiksel tabanlı yapay zeka yöntemleri veya yine istatistik tabanlı değerlendirme çeşitleri (raporlama gibi) için kullandığımız bir kütüphane. Veri üzerinde değişiklikler, parçalama, makine öğrenimi modelleri eğitimi gibi çeşitli amaçlar için kullanıyoruz.

Sklearn ile şunları yapacağız:
- Önceden oluşturduğumuz bir veri setini normalize edeceğiz.
- Bu veri setini eğitim, test ve doğrulama veri setlerine ayıracağız.
- Çok basit bir model eğitip raporlamasını yapacağız.

## Ödev

Bu hafta ödev olarak ise sizden bir "Veri Seti Okuma/Oluşturma Class'ı" yazmanızı istiyoruz. Bunu yaparken hem Python bilgilerinizi pekiştirmenizi hem de bu hafta işlediklerimizi kullanmanızı bekliyor olacağız. 

Yapmanız gerekenler:
- "DataOperations" adlı bir class oluşturmanız.
- Bu class'ta şunların olmasını bekliyoruz:
  - Eğer class oluşturulurken bir numpy array'i verilirse bunu pandas dataframe'ine çevirerek kaydetmesi,
  - Eğer bir path verilirse, ve bu path'de csv ya da json dosyası varsa bunu veriyi bir pandas dataframe'ine kaydetmesi,
  - Eğer direk olarak pandas dataframe'i verilirse de herhangi bir değişiklik yapmadan kaydetmesi,
  - Eğer herhangi bir değer verilmeden class oluşturulursa, rastgele bir veri seti oluşturmanız gerekmekte.
  - Class oluşturulduktan sonra ise bu veri üzerine analiz çıkartabilecek fonksiyonlar yazmanızı bekliyoruz:
    - Her sütundaki veri sayısı ve tipi, eğer sayısal veriler varsa standart sapma, ortalama gibi istatistiksel değerlerini veren bir fonksiyon,
    - Sayısal veriler içeren sütunların görselleştirilmesi.
- Tüm fonksiyonlara ve ana class içeriğine `Docstring`lerini yazmaya, isimlendirmeleri Python kurallarına göre yapmaya dikkat etmeye çalışın lütfen.